# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [19]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [20]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [21]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [22]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [23]:
def foldl2(f, x0, lst):
    return foldr(lambda val, acc: lambda n: acc(f(n, val)), lambda x: x, lst)(x0)

In [24]:
def foldr2(f, x0, lst):
    return foldl(lambda acc, val: lambda n: acc(f(val, n)), lambda x: x, lst)(x0)

In [25]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [26]:
import string

def check_inv(a, b):
    b_dict = dict.fromkeys(string.ascii_letters, 0)
    a_dict = dict.fromkeys(string.ascii_letters,0)
    for char in b: 
        b_dict[char] += 1
    for char in a[:len(b)]: 
        a_dict[char] += 1
    for i in range(len(b), len(a)):
        a_dict[a[i]] += 1
        a_dict[a[i - len(b)]] -= 1
        if(a_dict == b_dict):
            return True
    return False

check_inv('abcrotm', 'tro')

True

##### Для словарей: 
##### Получение элемента: O(1).
##### Установка элемента: O(1).
##### Удаление элемента: O(1).
##### Проверка на равенство словарей O(1) (от константы которая зависит от длинны алфавита)
##### Учитывая цикл получаем сложность O(n)
где n = len(a)



 

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [27]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            for k in self.left:
                yield k
        if (not self.left and not self.right):
            yield self.value
        if self.right:
            for k in self.right:
                yield k
    
    def __str__(self):
        string = str(self.value)
        if(self.left):
            string += ' ' + self.left.__str__()

        if(self.right):
            string += ' ' + self.right.__str__()      
        return string
    
    def __repr__(self):
        if(self.left):
            return f"Tree({self.value}, None, {self.right.__repr__()})"

        if(self.right):
            return f"Tree({self.value}, {self.left.__repr__()}, None)"
        
        if(not self.left and not self.right):
            return f"Tree({self.value})"

        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [28]:
print(tree)

0 1 3 4 2


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [29]:
import re

def calc_basic_str(char_list):
    result = 0
    s = ''.join(char_list)
    prep_list = re.split('(\+|\-|\*|\/)', s)
    operations = {
        "+": lambda x, y: x + y,
        "-": lambda x, y: x - y,
        "*": lambda x, y: x * y,
        "/": lambda x, y: x / y
    }
    operations_order = ['*', '/', '-', '+']
    for operation in operations_order:
        new_prep_list = []
        for i in range(len(prep_list)):
            if i==len(prep_list):
                break
            if (prep_list[i]==operation):
                new_prep_list.pop()
                new_prep_list.append(operations[operation](int(prep_list[i-1]), int(prep_list[i+1])))
                del prep_list[i+1]
            else: 
                new_prep_list.append(prep_list[i])
        prep_list = new_prep_list
    return prep_list[0]

def calc(expr):
    symbols = []
    for char in expr:
        if(char!=' '):
            symbols.append(char)
    left_brackets_ind = [i for i, j in enumerate(symbols) if j == '(']
    right_brackets_ind = [i for i, j in enumerate(symbols) if j == ')']

    for i in range(len(left_brackets_ind)):
        new_symbols = symbols[0:left_brackets_ind[i]] + [str(calc_basic_str(symbols[left_brackets_ind[i] + 1 :right_brackets_ind[-i]]))]+symbols[right_brackets_ind[-i] + 1 :]
    return calc_basic_str(new_symbols)
        
calc('2 * (15 - 3 * 4) - 2') == 4

True